In [33]:
import pandas as pd
import pandas
import numpy as np
import matplotlib.pyplot as plt
import pywt
from sklearn.model_selection import train_test_split
import scipy
import scipy.fftpack
from sklearn.decomposition import PCA
import pickle


In [34]:
testSet = pd.read_csv("test.csv", error_bad_lines=False)
testSet

,161,165,166,168,172,181,193,199,198,195,188,182,178,174,172.1,172.2,172.3,172.4,174.1,180,188.1,192,192.1,188.2
0,103,101,97,96,101,102,96,84,85,92,103,107,110,114,115,114,111,109,102,95,92,86,86,82
1,148,157,157,153,154,164,181,198,210,220,228,238,233,223,220,218,220,219,216,210,205,197,186,172
2,212,209,200,197,201,202,205,208,216,217,214,204,191,200,206,206,211,207,201,200,199,193,188,187
3,108,111,109,107,105,105,112,133,158,181,184,193,200,202,201,198,195,194,194,195,196,195,194,193
4,136,132,133,143,154,160,162,169,172,176,188,197,192,187,185,185,188,199,219,210,182,176,177,185
5,142,147,153,153,160,174,173,183,186,198,206,214,232,244,240,242,243,240,239,240,241,241,248,247
6,95,99,103,106,106,105,104,104,106,112,120,126,133,140,145,148,151,153,154,154,155,157,157,157
7,301,291,285,279,280,276,258,251,245,238,227,215,207,204,199,193,185,175,173,169,165,158,152,149
8,179,183,189,195,201,204,206,209,214,218,216,217,217,221,226,235,247,248,241,231,228,226,225,230
9,120,118,109,100,98,92,80,72,65,61,61,59,58,53,48,50,54,61,74,101,128,148,165,185


In [35]:
def extract_cgm_velocity(df, result_df):
  velocityDF = pd.DataFrame()
  for i in range(0,df.shape[1]-5):
      velocityDF['Vel_'+str(i)] = (df.iloc[:,i+5]-df.iloc[:,i])
  result_df['Window_Velocity_Max']=velocityDF.max(axis = 1, skipna=True)

def extract_cgm_trend(df, result_df):
    lunch = [[]]
    means = []
    for i in range(1, len(df)):
        lunch.append(df.iloc[i])
    for i in range(0, len(lunch)):
        means.append(df.iloc[i].mean())
    countmaster = []
    for i in range(0, len(lunch)):
        count = 0
        for j in df.iloc[i]:
            if j < means[i]:    
                count += 1
        countmaster.append(count)
    percentage=[]
    for i in countmaster:
        percentage.append((i / len(df.iloc[0])) * 100)
    result_df['cgmTrend'] = np.asarray(percentage)

#Extract Accelaration
def extract_acceleration(df, result_df):
    d=[]
    q=[]
    f=[]
    acc = [[],[],[],[]]
    for j in range(0, df.shape[0]):
        b = df.iloc[j]
        d = []
        for i in range(len(b)):
            if(np.isnan(b[i])):
                continue
            else:
                d.append(b[i])

        if(len(d) >= 1):
            solar_elevation_angle_1stdev = np.gradient(d)
            solar_elevation_angle_2nddev = np.gradient(solar_elevation_angle_1stdev)

            q = solar_elevation_angle_2nddev
            arr = q[5:10]
            acc[0].append(np.mean(arr))
            arr = q[10:15]
            acc[1].append(np.mean(arr))
            arr = q[15:20]
            acc[2].append(np.mean(arr))
            arr = q[20:25]
            acc[3].append(np.mean(arr))

        else:
            for i in range(4):
              acc[i].append(0)
    for i in range(4):
      result_df['acc'+str(i+1)] = acc[i]

#Extract entropy
def get_entropy(series):
    series_counts = series.value_counts()
    entropy = scipy.stats.entropy(series_counts)  
    return entropy

def extract_entropy(df, result_df):
    result_df['Entropy'] = df.apply(lambda row: get_entropy(row), axis=1)

#Extract Windowed mean
def extract_windowed_mean(df, result_df):
  if(df.shape[1] > 24):
    for i in range(6,df.shape[1],6):
      result_df['Mean_'+str(i-6)] = df.iloc[:,i:i+6].mean(axis = 1)
  else:
    for i in range(0,df.shape[1],6):
      result_df['Mean_'+str(i)] = df.iloc[:,i:i+6].mean(axis = 1)

# FFT- Finding top 8 values for each row
from numpy.fft import fft
def get_fft(row):
    cgmFFTValues = np.abs(fft(row))
    cgmFFTValues.sort()
    return np.flip(cgmFFTValues)[0:8]

def extract_fft(df, result_df):
  FFT = pd.DataFrame()
  FFT['FFT_Top2'] = df.apply(lambda row: get_fft(row), axis=1)
  FFT_updated = pd.DataFrame(FFT.FFT_Top2.tolist(), columns=['FFT_1', 'FFT_2', 'FFT_3', 'FFT_4', 'FFT_5', 'FFT_6', 'FFT_7', 'FFT_8'])
  FFT_updated.head()
  for i in range(8):
    result_df['FFT_'+str(i+1)] = FFT_updated['FFT_'+str(i+1)]

#1. Feature extraction
test_features = pd.DataFrame()

def extract_features(data, result_df) : 
    extract_cgm_velocity(data, result_df)
    extract_windowed_mean(data, result_df)
    extract_cgm_trend(data, result_df)
    extract_fft(data, result_df)
    extract_acceleration(data, result_df)
    extract_entropy(data, result_df)

extract_features(testSet, test_features)

test_features

,Window_Velocity_Max,Mean_0,Mean_6,Mean_12,Mean_18,cgmTrend,FFT_1,FFT_2,FFT_3,FFT_4,FFT_5,FFT_6,FFT_7,FFT_8,acc1,acc2,acc3,acc4,Entropy
0,29,100.000000,94.500000,112.166667,90.500000,45.833333,2383.0,116.056963,116.056963,101.480753,101.480753,40.439480,40.439480,29.461840,1.60,-1.85,-0.75,-0.0625,2.773718
1,66,155.500000,212.500000,222.166667,197.666667,37.500000,4727.0,455.118132,455.118132,141.939522,141.939522,47.328861,47.328861,34.925259,-0.10,-2.05,-0.95,-2.1875,2.925203
2,19,203.500000,210.666667,203.500000,194.666667,50.000000,4874.0,86.274952,86.274952,67.891227,67.891227,49.680921,49.680921,30.528675,-1.20,1.30,-1.00,0.5625,2.925203
3,81,107.500000,160.166667,198.333333,194.500000,37.500000,3963.0,593.414483,593.414483,241.754606,241.754606,90.266273,90.266273,85.941522,1.65,-2.40,0.60,-0.3750,2.787876
4,36,143.000000,177.333333,189.333333,191.500000,37.500000,4207.0,318.547875,318.547875,148.238390,148.238390,89.688349,89.688349,71.941742,0.60,-1.80,-3.60,6.8750,2.925203
5,58,154.833333,193.333333,240.166667,242.666667,45.833333,4986.0,550.684195,550.684195,175.178193,175.178193,157.977208,157.977208,94.707499,0.10,-1.75,0.10,-0.9375,2.925203
6,34,102.333333,112.000000,145.000000,155.666667,50.000000,3090.0,348.947910,348.947910,89.278768,89.278768,71.630999,71.630999,68.952433,1.55,-0.70,-0.50,-0.2500,2.787876
7,-23,285.333333,239.000000,193.833333,161.000000,54.166667,5275.0,665.785252,665.785252,301.714118,301.714118,188.264229,188.264229,146.164291,-0.80,0.80,0.30,0.6250,3.178054
8,31,191.833333,213.333333,232.333333,230.166667,41.666667,5206.0,256.638952,256.638952,113.603995,113.603995,82.054860,82.054860,51.883646,-0.65,1.70,-2.65,2.7500,3.062529
9,111,106.166667,66.333333,54.000000,133.500000,54.166667,2160.0,559.933780,559.933780,233.931631,233.931631,129.442976,129.442976,73.082146,1.00,0.45,4.90,-1.1250,3.040727


In [36]:

# # PCA
pca = PCA(n_components=10)
#Fit meal features
principalComponents = pca.fit(test_features)
#Transform meal features
testData = pca.fit_transform(test_features)
testData

array([[ 1.54735312e+03,  3.93792904e+02,  8.51775861e+01,
         1.91963377e+01, -1.25785376e+01, -2.34963652e+01,
         1.31561509e+01, -3.30652634e+00,  1.04779764e+00,
         8.27720361e-02],
       [-8.08972855e+02, -3.28398437e+01, -9.18395330e+01,
         3.55678845e+01, -5.83165748e+01,  1.70862705e+01,
         1.07941223e+01,  4.38534572e+00,  8.72298288e-01,
         3.58291094e-01],
       [-9.49920385e+02,  4.87840627e+02,  2.30220449e+01,
        -1.15085054e+01, -3.21379587e+00,  2.52193536e+01,
        -3.72530937e+00,  2.70283032e-01, -4.65508366e+00,
        -9.98325599e-01],
       [-4.79586510e+01, -2.86299123e+02, -2.80060143e+01,
         8.37430156e+01, -1.18886066e+01, -2.27725773e+01,
        -9.09659706e+00, -4.88437403e+00, -2.61665576e+00,
        -4.65465645e-01],
       [-2.87073436e+02,  1.22486018e+02,  1.71065957e+01,
         1.68226633e+01,  1.92147978e+01, -1.60014012e+01,
        -8.10300239e+00,  8.07287473e+00,  6.94134453e+00,
        -4.

In [45]:
clf = pickle.load(open('89_decision_tree.pkl', 'rb'))
y_pred = clf.predict(testData)
new_y_pred = []
for i in y_pred:
    if(i > 0):
        new_y_pred.append(1)
    else:
        new_y_pred.append(0)
        
result = pd.DataFrame(new_y_pred)
result.to_csv('Result.csv', index=False)

[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1]
Correct prob: 0.8181818181818182
